# Residual value model

## Imports

In [97]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import ast

## 1.- Load data

In [98]:
makes=["acura","audi","bmw","buick","cadillac","chevrolet","chrysler","dodge","ford","gmc","honda","hyundai","infiniti","jeep","kia","land_rover","lexus",
     "lincoln","mazda","mercedes_benz","mini","mitsubishi","nissan","porsche","ram","subaru","tesla","toyota","volkswagen","volvo"]
df_list=[]
for make in makes:
    for batch in range(1,26):
        path='cars\\'+make+'\\cars_'+make+'_'+str(batch)+'.json'
        if os.path.exists(path):
            df_list.append(pd.read_json(path))

df=pd.concat(df_list, ignore_index=True,axis=0)
df.head()

,year_manufacture,years,make,model,mileage,stock_type,interior_color,exterior_color,drive_train,mpg,fuel_type,transmission,engine,price_USD,url
0,2006,19,acura,Acura TSX Base,76497,Used,Biege,Green Pearl,Front-wheel Drive,22–31 Based on EPA mileage ratings. Use for co...,Gasoline,Automatic,"2.4L I-4 DOHC, i-VTEC variable valve control, ...",10995.0,https://cars.com/vehicledetail/d6115b1a-3830-4...
1,2021,4,acura,Acura RDX Technology Package,54931,Used,Espresso,Majestic Black Pearl,All-wheel Drive,21–27 Based on EPA mileage ratings. Use for co...,Gasoline,10-Speed Automatic,"2L I-4 gasoline direct injection, DOHC, VTEC v...",27985.0,https://cars.com/vehicledetail/f9548b93-31b2-4...
2,2017,8,acura,Acura MDX 3.5L w/Technology Package,103720,Used,Ebony,Modern Steel Metallic,All-wheel Drive,18–26 Based on EPA mileage ratings. Use for co...,Gasoline,Automatic,"3.5L V-6 gasoline direct injection, i-VTEC var...",17981.0,https://cars.com/vehicledetail/06e7a2fc-13ec-4...
3,2024,1,acura,Acura Integra A-SPEC,17309,Used,Ebony,Platinum White Pearl,Front-wheel Drive,29–36 Based on EPA mileage ratings. Use for co...,Gasoline,Automatic,"1.5L I-4 gasoline direct injection, DOHC, VTEC...",30049.0,https://cars.com/vehicledetail/658c133c-dc51-4...
4,2017,8,acura,Acura RDX Technology & AcuraWatch Plus Package,66552,Used,Ebony,White Diamond Pearl,All-wheel Drive,19–27 Based on EPA mileage ratings. Use for co...,Gasoline,Automatic,"3.5L V-6 i-VTEC variable valve control, premiu...",18950.0,https://cars.com/vehicledetail/79565df9-1fba-4...


## 2.- Data Exploration and Preprocesing

### 2.1.- Removing duplicates
Durante el proceso de extracción de datos, pueden surgir duplicidades. Se verifica su existencia y, en caso afirmativo, se eliminan las filas duplicadas del DataFrame.<br> <br>
During the data extraction process, duplicates may arise. Their presence is verified, and if detected, duplicate rows are removed from the DataFrame.

In [99]:
num_duplicated_rows=sum(df.duplicated())
print(f'The number of duplicated rows is {num_duplicated_rows}')

The number of duplicated rows is 8982


In [100]:
df = df.drop_duplicates()
num_duplicated_rows=sum(df.duplicated())
print(f'The number of duplicated rows is now {num_duplicated_rows}')

The number of duplicated rows is now 0


### 2.2.- Variable description

A continuación, se realiza un análisis de alto nivel de las variables disponibles, utilizando un diccionario de datos y evaluando la cantidad de valores ausentes, valores atípicos o categorías.

#### Diccionario de datos
- **year_manufacture**: Año de fabricación del vehículo, que generalmente coincide con el año de compra.
- **years**: Antigüedad del vehículo en años, calculada en 2025. Se obtiene como la diferencia entre el año 2025 y el año de fabricación.
- **make**: Marca del vehículo.
- **model**: Modelo del vehículo.
- **mileage**: Kilometraje del vehículo, expresado en millas.
- **stock_type**: Indicador de si el vehículo es nuevo o usado.
- **interior_color**: Color del interior del vehículo, expresado en lenguaje natural.
- **exterior_color**: Color del exterior del vehículo, expresado en lenguaje natural.
- **drive_train**: Tipo de tracción del vehículo (delantera, trasera, a las cuatro ruedas).
- **mpg**: Consumo de combustible, expresado en millas por galón. Puede incluir información sobre la diferencia entre el consumo en ciudad y en carretera.
- **fuel_type**: Tipo de motorización (diésel, gasolina, híbrido, eléctrico, etc.).
- **transmission**: Tipo de transmisión. Cadena de texto que indica si es manual o automática y el número de marchas. Transmission type.
- **engine**: Cadena de caracteres que describe el motor en lenguaje natural. Puede incluir información sobre la cilindrada y el volumen. Text string that describes the engine in natural language.
- **price_USD**: Precio de venta del vehículo a la fecha de extracción de datos, expresado en dólares estadounidenses.
- **url**: Dirección web de donde se ha extraído la información.
  
Datos obtenidos en la página web [Cars.com](https://www.cars.com/) entre los días 22/03/2025 y 28/03/2025.

Next, a high-level analysis of the available variables is conducted using a data dictionary and assessing the number of missing values, outliers, or categories.
#### Data dictionary
- **year_manufacture**: Year of vehicle manufacture, which generally corresponds to the year of purchase.
- **years**: Vehicle age in years, calculated in 2025. It is obtained as the difference between the year 2025 and the year of manufacture.
- **make**: Vehicle make.
- **model**: Vehicle model.
- **mileage**: Vehicle mileage, expressed in miles.
- **stock_type**: Indicator of whether the vehicle is new or used.
- **interior_color**: Interior color of the vehicle, expressed in natural language.
- **exterior_color**: Exterior color of the vehicle, expressed in natural language.
- **drive_train**: Vehicle drivetrain type (front, rear, all-wheel drive).
- **mpg**: Fuel consumption, expressed in miles per gallon. It may include information about the difference between city and highway consumption.
- **fuel_type**: Engine type (diesel, gasoline, hybrid, electric, etc.).
- **transmission**: Transmission type. Text string indicating whether it is manual or automatic and the number of gears.
- **engine**: Text string that describes the engine in natural language. It may include information about the engine displacement and volume.
- **price_USD**: Vehicle sale price at the data extraction date, expressed in US dollars.
- **url**: Web address from which the information was extracted.

Data obtained from the website [Cars.com](https://www.cars.com/) between the dates 03/22/2025 and 03/28/2025.

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 141185 entries, 0 to 150166
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   year_manufacture  141185 non-null  int64  
 1   years             141185 non-null  int64  
 2   make              141185 non-null  object 
 3   model             141185 non-null  object 
 4   mileage           141185 non-null  int64  
 5   stock_type        141185 non-null  object 
 6   interior_color    141185 non-null  object 
 7   exterior_color    141185 non-null  object 
 8   drive_train       141185 non-null  object 
 9   mpg               131855 non-null  object 
 10  fuel_type         133860 non-null  object 
 11  transmission      141185 non-null  object 
 12  engine            141185 non-null  object 
 13  price_USD         141181 non-null  float64
 14  url               141185 non-null  object 
dtypes: float64(1), int64(3), object(11)
memory usage: 17.2+ MB


#### Check the Categorical and Numerical Columns

In [102]:
# Categorical variables
cat_col = [col for col in df.columns if df[col].dtype == 'object']
print('Categorical variables:',cat_col)
# Numerical variables
num_col = [col for col in df.columns if df[col].dtype != 'object']
print('Numerical variables:',num_col)

Categorical variables: ['make', 'model', 'stock_type', 'interior_color', 'exterior_color', 'drive_train', 'mpg', 'fuel_type', 'transmission', 'engine', 'url']
Numerical variables: ['year_manufacture', 'years', 'mileage', 'price_USD']


In [103]:
#Number of unique values for categorical variables
df[cat_col].nunique()

make                  30
model               9201
stock_type            26
interior_color      4507
exterior_color      5674
drive_train           17
mpg                  931
fuel_type             29
transmission        1019
engine              5278
url               141027
dtype: int64

In [104]:
for col in cat_col:
    print(col)
    print('Sample of some unique values:', df[col].unique()[:20],'\n')

make
Sample of some unique values: ['acura' 'audi' 'bmw' 'buick' 'cadillac' 'chevrolet' 'chrysler' 'dodge'
 'ford' 'gmc' 'honda' 'hyundai' 'infiniti' 'jeep' 'kia' 'land_rover'
 'lexus' 'lincoln' 'mazda' 'mercedes_benz'] 

model
Sample of some unique values: ['Acura TSX Base' 'Acura RDX Technology Package'
 'Acura MDX 3.5L w/Technology Package' 'Acura Integra A-SPEC'
 'Acura RDX Technology & AcuraWatch Plus Package' 'Acura TLX V6 Tech'
 'Acura Integra A-SPEC w/ Technology' 'Acura TLX V6' 'Acura ZDX A-SPEC'
 'Acura MDX Sport Hybrid SH-AWD Sport Hybrid w/Technology Pkg'
 'Acura RLX Technology Package' 'Acura MDX SH-AWD Technology'
 'Acura TLX Base' 'Acura MDX SH-AWD' 'Acura ZDX A-Spec'
 'Acura RDX SH-AWD Technology' 'Acura MDX Sport Hybrid Technology Package'
 'Acura TSX Technology' 'Acura ILX Base' 'Acura MDX 3.5L'] 

stock_type
Sample of some unique values: ['Used' 'Acura Certified' 'Certified' 'Audi Certified' 'BMW Certified'
 'Buick Certified' 'Cadillac Certified' 'Chevrolet Certified

#### Feature engineering

##### MPG

Se puede observar que este campo está compuesto por un par de valores numéricos separados por un guion, seguidos de una nota informativa: 'Based on EPA mileage ratings. Use for comparison purposes only. Actual mileage will vary depending on driving conditions, driving habits, vehicle maintenance, and other factors.' Asumiremos que, en producción, se contará con los 'EPA mileage ratings' de los vehículos, por lo que esta variable puede seguir siendo útil. De ella, obtendremos los valores de consumo en ciudad y carretera si están disponibles.

It can be observed that this field consists of a pair of numerical values separated by a hyphen, followed by an informational note: 'Based on EPA mileage ratings. Use for comparison purposes only. Actual mileage will vary depending on driving conditions, driving habits, vehicle maintenance, and other factors.' We will assume that, in production, the 'EPA mileage ratings' of the vehicles will be available, so this variable may still be useful. From it, we will obtain the values for city and highway consumption if available.

In [105]:
def keep_numbers_and_en_dash(text):
    if text is None:
        return None  # Return None if the value is None
    return re.sub(r'[^0-9–]', '', text)  # Keep numbers and en dash (–)
def get_mpg_city(text):
    if text is None:
        return None  # Return None if the value is None
    if text == "–":
        return None
    if "–" not in text:
        return float(text) if text.replace('.', '', 1).isdigit() and text.count('.') < 2 else None
    return float(text.split('–')[0]) if text.split('–')[0].replace('.', '', 1).isdigit() and text.split('–')[0].count('.') < 2 else None

def get_mpg_highway(text):
    if text is None:
        return None  # Return None if the value is None
    if text == "–":
        return None
    if "–" not in text:
        return float(text) if text.replace('.', '', 1).isdigit() and text.count('.') < 2 else None
    return float(text.split('–')[1]) if text.split('–')[1].replace('.', '', 1).isdigit() and text.split('–')[1].count('.') < 2 else None

df.loc[df['mpg'].apply(lambda x: isinstance(x, float) and np.isnan(x)),'mpg']=None # substitue NaN for None
df['auxiliar'] = df['mpg'].apply(keep_numbers_and_en_dash)
df['mpg_city'] = df['auxiliar'].apply(get_mpg_city)
df['mpg_highway'] = df['auxiliar'].apply(get_mpg_highway)



In [106]:
# Check nulls in mpg_city: All null values in mpg_city are due to the fact that mpg is None or '–'
df[df['mpg_city'].isnull()]['mpg'].unique()

array([None, '–'], dtype=object)

In [107]:
# Check nulls in mpg_highway: All null values in mpg_highway are due to the fact that mpg is None or '–'
df[df['mpg_highway'].isnull()]['mpg'].unique()

array([None, '–'], dtype=object)

In [108]:
df.drop(columns=['mpg', 'auxiliar'], inplace=True)
df.head()

,year_manufacture,years,make,model,mileage,stock_type,interior_color,exterior_color,drive_train,fuel_type,transmission,engine,price_USD,url,mpg_city,mpg_highway
0,2006,19,acura,Acura TSX Base,76497,Used,Biege,Green Pearl,Front-wheel Drive,Gasoline,Automatic,"2.4L I-4 DOHC, i-VTEC variable valve control, ...",10995.0,https://cars.com/vehicledetail/d6115b1a-3830-4...,22.0,31.0
1,2021,4,acura,Acura RDX Technology Package,54931,Used,Espresso,Majestic Black Pearl,All-wheel Drive,Gasoline,10-Speed Automatic,"2L I-4 gasoline direct injection, DOHC, VTEC v...",27985.0,https://cars.com/vehicledetail/f9548b93-31b2-4...,21.0,27.0
2,2017,8,acura,Acura MDX 3.5L w/Technology Package,103720,Used,Ebony,Modern Steel Metallic,All-wheel Drive,Gasoline,Automatic,"3.5L V-6 gasoline direct injection, i-VTEC var...",17981.0,https://cars.com/vehicledetail/06e7a2fc-13ec-4...,18.0,26.0
3,2024,1,acura,Acura Integra A-SPEC,17309,Used,Ebony,Platinum White Pearl,Front-wheel Drive,Gasoline,Automatic,"1.5L I-4 gasoline direct injection, DOHC, VTEC...",30049.0,https://cars.com/vehicledetail/658c133c-dc51-4...,29.0,36.0
4,2017,8,acura,Acura RDX Technology & AcuraWatch Plus Package,66552,Used,Ebony,White Diamond Pearl,All-wheel Drive,Gasoline,Automatic,"3.5L V-6 i-VTEC variable valve control, premiu...",18950.0,https://cars.com/vehicledetail/79565df9-1fba-4...,19.0,27.0


#### Transmission

De esta columna, queremos derivar otras dos columna: una que nos indique el tipo de transmisión (automática, manual, [manumática](https://es.wikipedia.org/wiki/Transmisi%C3%B3n_manum%C3%A1tica), [variable](https://es.wikipedia.org/wiki/Transmisi%C3%B3n_variable_continua) o [dual-clutch](https://es.wikipedia.org/wiki/Caja_de_cambios_de_doble_embrague) ), y otra que nos indique el número de marchas del vehículo.

From this column, we want to derive two other columns: one indicating the type of transmission (automatic, manual, [manumatic](https://en.wikipedia.org/wiki/Manumatic), [variable](https://en.wikipedia.org/wiki/Continuously_variable_transmission), or [dual-clutch](https://en.wikipedia.org/wiki/Dual-clutch_transmission)), and another indicating the number of gears/speeds of the vehicle.

In [109]:
def get_speeds_of_transmission(text):
    if text is None:
        return None  # Return None if the value is None
    if "SINGLE" in text.upper():
        return 1
    if "TWO" in text.upper():
        return 1
    if text in ('4L80E', '4l60e'):
        return 4
    if text in ('TH400', 'TH350', 'TH 400'):
        return 3
    if text == '6L70E':
      return 6

    substring=re.sub(r'[^0-9]', '', text)
    if substring=="":
        return None
    if len(substring)==1:
        return float(substring)
    if len(substring)>=2:
        if substring[0]=="1":
            if float(substring)==10006: #The general logic that the first numbers are the number of gears soesn't apply for Allison 1000 6-Speed Automatic 
                return 6
            if float(substring)==1503: # nor Borg-Warner T150 3 Speed Manual
                return 3
            return float(substring)
        else:
            return float(substring[0])
    return None

def keep_letters(text):
    return re.sub(r'[^a-zA-Z]', '', text)

def get_transmission_type(text):
    if "MANUAL" in text.upper() or "M/T" in text.upper() or "SMG" in text.upper(): # SMG for bmw
        return "Manual"
    if "AUTO" in text.upper() or "A/T" in text.upper() \
    or keep_letters(text).upper()=='A' or "AU" in text.upper() or "AT" in text.upper()\
    or '6L80' in text.upper() \
    or " A" in text.upper() or "ZF" in text.upper() or 'ECT' in text.upper() \
    or text in ('4L80E','TH400', 'TH350', 'TH 400', '4l60e', '6L70E', 'EFLITE SI-EVT', 'E4OD 4R100', '8HP75'): #ZF and 8HP75 for bmw, ECT for toyota, 4L80E, 6L70E,6L80, 4l60e for GM group, TH350 and TH400 for chevrolet and buick, EFLITE SI-EVT chrysler
        return "Automatic"
    if "VARIABLE" in text.upper() or "CVT" in text.upper() or text in ('i-VT', 'IVT'): #Continuously Variable Transmission, ivt for hyundai
        return "Variable"
    if "DUAL" in text.upper() or "DOUBLE" in text.upper() or "DC" in text.upper() or "S TRONIC" in text.upper() or "S-TRONIC" in text.upper() or "PDK" in text.upper(): # Dual-Clutch Transmission (DCT) stronic for audi, PDK for Porsche
        return "Dual-clutch"
    if "GEARTRONIC" in text.upper() or "SHIFTRONIC" in text.upper() \
    or "STEPTRONIC" in text.upper() or "TRIPTONIC" in text.upper() \
    or "TIPTRONIC" in text.upper() or "SPORTRONIC" in text.upper(): #Geartronic for volvo, steptronic for bmw, Tiptronic for volkswagen group, sportronic for Mitsubishi
        return "Manumatic"
    return None
df['num_speeds']=df['transmission'].apply(get_speeds_of_transmission)
df['transmission_type']=df['transmission'].apply(get_transmission_type)

In [110]:
# Check nulls in transmission_type: All null values in transmission_type are due to the fact that transmission variable doesn't give any clue of the transmission type
df[df['transmission_type'].isnull()]['transmission'].unique()

array(['Not Specified', '9-Speed', 'Single-speed transmission', '7 speed',
       '–', '6-Speed', '5-Speed', 'Single Speed', '3-Speed',
       'Transmission Overdrive Switch', 'Single-Speed Fixed Gear',
       'Single Reduction Gear', 'Unspecified', '10 Speed', 'Sequential',
       'SINGLE-SPEED TRANSMISSION', '10-SPEED TRANSMISSION',
       'NOT SPECIFIED', '448', '8', 'Standard', '8-Speed',
       'Single-Speed Reduction Gear', 'DFT', 'Single-Speed Fi', 'Other',
       '15S', 'MOD HYBRID TRANSMISSION', 'N/A',
       '6-Spd Sport Transmission', '5SP', '5 Speed',
       'Continuously Variabl', '9SP', 'Single Speed Reducer', '5M',
       '7-Speed', 'standard', '6 Speed', 'Single Speed Transmission', 'C',
       'SINGLE-SPEED FIXED GEAR', 'Single-Speed Fixed Gear Transmission',
       '4 Speed Transmission', '1-SPEED G', 'AWD', 'FWD', 'Drivetrai'],
      dtype=object)

In [111]:
# Check nulls in num_speeds: All null values in num_speeds are due to the fact that transmission variable doesn't indicate the number of gears/speeds
df[df['num_speeds'].isnull()]['transmission'].unique()

array(['Automatic', 'CVT', 'Manual', 'Variable', 'A/T',
       'Transmission w/Dual Shift Mode', 'Automatic CVT',
       'Continuously Variable Transmission', 'Not Specified',
       'Automatic w/OD', 'Auto-Shift Manual', 'CVT Transmission',
       'Continuously Variable', 'Transmission-Auto', 'AUTO',
       'Automatic with Tiptronic', 'quattroa? s tronica?', 'M/T',
       'quattroA? S tronicA?', 'quattroA? TiptronicA?', 'A', 'AUTOMATIC',
       'quattro, s-tronic', 'Automatic w/Tiptronic', '–',
       'quattro S tronic', 'CVT with Multitronic', 'FWD, s-tronic',
       'Auto, CVT Multitronic', 'Auto', 'Automatic w/Steptronic', 'auto',
       'Steptronic', 'Automatic w/Manual Shift', 'DCT',
       'STEPTRONIC AUTOMATIC', 'automatic', 'Automatic Automatic',
       'Dynaflow  Automatic', 'Automatic, CVT', 'AT',
       'Continuously Variable (CVT)', 'CONTINUOUSLY VARIABLE (CVT)',
       'AUTO Automatic', 'Transmission Overdrive Switch', 'CVT Automatic',
       'manual', 'Unspecified', 'CVT

In [112]:
df.head()

,year_manufacture,years,make,model,mileage,stock_type,interior_color,exterior_color,drive_train,fuel_type,transmission,engine,price_USD,url,mpg_city,mpg_highway,num_speeds,transmission_type
0,2006,19,acura,Acura TSX Base,76497,Used,Biege,Green Pearl,Front-wheel Drive,Gasoline,Automatic,"2.4L I-4 DOHC, i-VTEC variable valve control, ...",10995.0,https://cars.com/vehicledetail/d6115b1a-3830-4...,22.0,31.0,NaN,Automatic
1,2021,4,acura,Acura RDX Technology Package,54931,Used,Espresso,Majestic Black Pearl,All-wheel Drive,Gasoline,10-Speed Automatic,"2L I-4 gasoline direct injection, DOHC, VTEC v...",27985.0,https://cars.com/vehicledetail/f9548b93-31b2-4...,21.0,27.0,10.0,Automatic
2,2017,8,acura,Acura MDX 3.5L w/Technology Package,103720,Used,Ebony,Modern Steel Metallic,All-wheel Drive,Gasoline,Automatic,"3.5L V-6 gasoline direct injection, i-VTEC var...",17981.0,https://cars.com/vehicledetail/06e7a2fc-13ec-4...,18.0,26.0,NaN,Automatic
3,2024,1,acura,Acura Integra A-SPEC,17309,Used,Ebony,Platinum White Pearl,Front-wheel Drive,Gasoline,Automatic,"1.5L I-4 gasoline direct injection, DOHC, VTEC...",30049.0,https://cars.com/vehicledetail/658c133c-dc51-4...,29.0,36.0,NaN,Automatic
4,2017,8,acura,Acura RDX Technology & AcuraWatch Plus Package,66552,Used,Ebony,White Diamond Pearl,All-wheel Drive,Gasoline,Automatic,"3.5L V-6 i-VTEC variable valve control, premiu...",18950.0,https://cars.com/vehicledetail/79565df9-1fba-4...,19.0,27.0,NaN,Automatic


Later on we'll discuss how to handle missing data and impute values

In [113]:
df.groupby(['transmission_type','num_speeds'], dropna=False).size().reset_index(name='Count')

,transmission_type,num_speeds,Count
0,Automatic,1.0,2944
1,Automatic,2.0,177
2,Automatic,3.0,77
3,Automatic,4.0,1122
4,Automatic,5.0,1571
5,Automatic,6.0,9985
6,Automatic,7.0,3884
7,Automatic,8.0,15761
8,Automatic,9.0,6306
9,Automatic,10.0,4438


#### Drive train

In [114]:
def get_drive_train(text):
    if "FRONT" in text.upper() or 'FWD' == text:
        return "Front-wheel Drive"
    if "ALL" in text.upper() or "FOUR" in text.upper() or '4' in text or 'AWD'==text:
        return "All-wheel Drive"
    if "REAR" in text.upper() or 'RWD'==text:
        return "Rear-wheel Drive"
    return None
df['drive_train_v2']=df['drive_train'].apply(get_drive_train)


In [115]:
# Check nulls in drive_train_v2: All null values in drive_train_v2 are due to the fact that drive_train variable doesn't provide the information
df[df['drive_train_v2'].isnull()]['drive_train'].unique()

array(['–', '2WD', 'Unknown'], dtype=object)

In [116]:
df.head()

,year_manufacture,years,make,model,mileage,stock_type,interior_color,exterior_color,drive_train,fuel_type,transmission,engine,price_USD,url,mpg_city,mpg_highway,num_speeds,transmission_type,drive_train_v2
0,2006,19,acura,Acura TSX Base,76497,Used,Biege,Green Pearl,Front-wheel Drive,Gasoline,Automatic,"2.4L I-4 DOHC, i-VTEC variable valve control, ...",10995.0,https://cars.com/vehicledetail/d6115b1a-3830-4...,22.0,31.0,NaN,Automatic,Front-wheel Drive
1,2021,4,acura,Acura RDX Technology Package,54931,Used,Espresso,Majestic Black Pearl,All-wheel Drive,Gasoline,10-Speed Automatic,"2L I-4 gasoline direct injection, DOHC, VTEC v...",27985.0,https://cars.com/vehicledetail/f9548b93-31b2-4...,21.0,27.0,10.0,Automatic,All-wheel Drive
2,2017,8,acura,Acura MDX 3.5L w/Technology Package,103720,Used,Ebony,Modern Steel Metallic,All-wheel Drive,Gasoline,Automatic,"3.5L V-6 gasoline direct injection, i-VTEC var...",17981.0,https://cars.com/vehicledetail/06e7a2fc-13ec-4...,18.0,26.0,NaN,Automatic,All-wheel Drive
3,2024,1,acura,Acura Integra A-SPEC,17309,Used,Ebony,Platinum White Pearl,Front-wheel Drive,Gasoline,Automatic,"1.5L I-4 gasoline direct injection, DOHC, VTEC...",30049.0,https://cars.com/vehicledetail/658c133c-dc51-4...,29.0,36.0,NaN,Automatic,Front-wheel Drive
4,2017,8,acura,Acura RDX Technology & AcuraWatch Plus Package,66552,Used,Ebony,White Diamond Pearl,All-wheel Drive,Gasoline,Automatic,"3.5L V-6 i-VTEC variable valve control, premiu...",18950.0,https://cars.com/vehicledetail/79565df9-1fba-4...,19.0,27.0,NaN,Automatic,All-wheel Drive


#### Stock type

In [117]:
def stock_type(text):
    if "CERTIFIED" in text.upper():
        return "Certified"
    if "USED" in text.upper():
        return "Used"
    if "NEW" in text.upper():
        return "New"
    return None

df['stock_type_v2']=df['stock_type'].apply(stock_type)
df['stock_type_v2'].unique()

array(['Used', 'Certified'], dtype=object)

#### Vehicle model

In [118]:
# Open the file containing the dictionary
with open('model_vehicle_type.txt', 'r') as file:
    content = file.read()
# Safely evaluate the content as a Python dictionary
models_dict = ast.literal_eval(content)


In [119]:
def get_model(row):
    text=row['model'].upper()
    models =[model.upper() for model in list(models_dict[row['make']].keys())]
    for model in models:
        if model in text:
            return model
    if row['make']=='buick':
        return 'GRANSPORT'
    return None
      
df['model_v2'] = df.apply(get_model, axis=1)

In [120]:
# Check nulls in model_v2: There are not
df[(df['model_v2'].isnull())]['model'].unique()

array([], dtype=object)

In [121]:
num_models=len(df['model_v2'].unique())
print(f"The number of different car models considered is {num_models}")

The number of different car models considered is 704


#### Vehicle type

In [122]:
def get_vehicle_type(row):
    return models_dict[row['make']][row['model_v2']]
      
df['vehicle_type'] = df.apply(get_vehicle_type, axis=1)

In [123]:
df.groupby(['make','model_v2','vehicle_type'], dropna=False).size().reset_index(name='Count').to_excel('vehicle_types.xlsx', index=False)

In [124]:
df.groupby(['make','model_v2','vehicle_type'], dropna=False).size().reset_index(name='Count').sort_values(by='Count', ascending=False).head(50)

,make,model_v2,vehicle_type,Count
573,mitsubishi,OUTLANDER,SUV,2754
619,ram,1500,Truck,2600
219,chrysler,PACIFICA,SUV,2161
455,land_rover,RANGE ROVER,SUV,2115
561,mini,HARDTOP,Compact,1762
559,mini,COUNTRYMAN,SUV,1726
103,buick,ENCORE,SUV,1687
501,mazda,CX-5,SUV,1687
4,acura,MDX,SUV,1578
427,jeep,WRANGLER,SUV,1491


#### Engine displacement

In [125]:
def get_left_hand_side(text):
    text=text.replace(',','')
    result=""
    already_found_dot=False
    for char in text[::-1]:
        if char.isdigit():
            result = char + result
        elif char == '.' and not already_found_dot:
            result = char + result
            already_found_dot=True   
        else:
            break  # Stop when we encounter a non-number, non-dot character, or a second dot
    return result

def get_engine_displacement(text):
    if text.replace('.', '', 1).isdigit() and text.count('.') < 2 and float(text)<10:
        return float(text)
    possible_splitters=['CC', 'cc', '-liter', 'liter', 'T','I','CI', 'ci','c.i.', '/','L ','L', 'l ', ' ']
    for splitter in possible_splitters:
        for i in range(text.count(splitter)):
            liters=get_left_hand_side(text.split(splitter)[i].strip())
            if liters.replace('.', '', 1).isdigit() and liters.count('.')< 2:
                if splitter in ('CC', 'cc'):
                    return float(liters)/1000 
                elif splitter in ('CI', 'ci', 'c.i.'):
                    return float(liters)/61.024
                elif splitter == ' ':
                    if '.' in liters and float(liters) < 10:
                        return float(liters)
                elif splitter in ('T', 'I', '/'):
                    if float(liters) < 10:
                        return float(liters)
                else:
                    if float(liters) < 10:
                        return float(liters)
    return None
df['engine_displacement']=df['engine'].apply(get_engine_displacement)

In [126]:
# Check nulls in engine_displacement: All null values in engine_displacement are due to the fact that engine variable doesn't provide that information
df[df['engine_displacement'].isnull()]['engine'].unique()


array(['Electric ZEV 490hp', 'Gas', 'Electric', 'Turbo Gas',
       '4 Cylinder Engine', '–', '4 Cylinder', 'Electric ZEV 358hp',
       '6 Cylinder', 'Electric ZEV 499hp', 'Electric Motor', 'V6', 'I4',
       'Engine: Dual Motor -inc: start/stop pushbutton', 'I-4 cyl',
       'V6 Cylinder Engine', 'Electric ZEV', 'Supercharged Gas',
       '355.0HP Electric Motor Electric Fuel System',
       'Electric 402hp 490ft. lbs.',
       'Dual Synchronous Electric Motors Engine', 'L Electric Motor',
       'L 4-Cyl Engine', '0 Cylinders',
       'Dual Asynchronous Electric Motors Engine',
       'Dual AC Electric Motors', 'Plug-in Hybrid', 'Not Specified', 'V8',
       'I6', 'Range-Extended Electric 168hp 184ft. lbs.',
       'Straight 6 Cylinder Engine', 'Electric LEV3-SULEV30 170hp',
       'ELECTRIC', '8 Cylinder Engine', 'AC Electric Motor',
       'Turbo Diesel', '39.5L Electric Motor', '8 Cylinder',
       'ELECTRIC MOTOR', 'Electric 536hp 549ft. lbs.',
       ': 5th Generation Electric 

#### Engine cylinders

In [127]:
df['engine'].unique()[:50]

array(['2.4L I-4 DOHC, i-VTEC variable valve control, premium unleaded,',
       '2L I-4 gasoline direct injection, DOHC, VTEC variable valve cont',
       '3.5L V-6 gasoline direct injection, i-VTEC variable valve contro',
       '1.5L I-4 gasoline direct injection, DOHC, VTEC variable valve co',
       '3.5L V-6 i-VTEC variable valve control, premium unleaded, engine',
       '2.4L I4', '3.5L V6 24V GDI SOHC',
       '3.5L V-6 gasoline direct injection, variable valve control, prem',
       'Electric ZEV 490hp',
       '3.0L PGM-FI 24V SOHC i-VTEC V6 -inc: Variable Cylinder Managemen',
       'Gas', '2.0L I4 16V GDI DOHC Turbo', '3.5L V6 SOHC 24V',
       'Electric', 'Turbo Gas',
       '3L V-6 i-VTEC variable valve control, premium unleaded, engine w',
       '2.4L I-4 gasoline direct injection, DOHC, i-VTEC variable valve',
       '3.5L V6 24V MPFI SOHC',
       '3.2L V-6 variable valve control, premium unleaded, engine with 2',
       '3.5L V-6 variable valve control, premium unle

In [128]:
def get_right_hand_side_cylinders(text):
    result=""
    for char in text:
        if char.isdigit():
            result += char
        elif char =='.':
            return "" #This number has nothing to do with cylinders
        elif char =='V':
            return "" #This is the number of valves
        else:
            break  # Stop when we encounter a non-number, non-dot character, or a second dot
    return result

def get_left_hand_side_cylinders(text):
    result=""
    for char in text[::-1]:
        if char.isdigit():
            result = char + result
        elif char =='.':
            return "" #This number has nothing to do with cylinders
        else:
            break  # Stop when we encounter a non-number, non-dot character, or a second dot
    return result

def get_engine_cylinders(text):
    possible_splitters=['Inline-','Flat-', 'Inline','Flat','Transverse','I-', 'V-', 'W-', 'H-', 'L-', 'l-', 'IC', 'VR', 'I', 'H', 'V',  'T', 'L', 'F']
    for splitter in possible_splitters:
        for i in range(text.count(splitter)):
            cylinders=get_right_hand_side_cylinders(text.split(splitter)[i+1].strip())

            if cylinders.isdigit() and 0<=int(cylinders)<=16 and "VALVE" not in text.split(splitter)[i+1].upper()[:9]:
                return int(cylinders)
            
    possible_splitters=['-CYL', 'CYL']
    for splitter in possible_splitters:
        for i in range(text.upper().count(splitter)):
            cylinders=get_left_hand_side_cylinders(text.upper().split(splitter)[i].strip())
            if cylinders.isdigit() and 0<=int(cylinders)<=16:
                return int(cylinders) 
    return None
df['engine_cylinders']=df['engine'].apply(get_engine_cylinders)

In [129]:
print("There are",str(len(df[df['engine_cylinders'].isnull()]['engine'])), "cars in which the number of cylinders hasn't been obtained")

df[df['engine_cylinders'].isnull()]['engine'].unique()

There are 16008 cars in which the number of cylinders hasn't been obtained


array(['Electric ZEV 490hp', 'Gas', 'Electric', 'Turbo Gas', '–',
       '2.4L DOHC 16V', 'Electric ZEV 358hp', '2.0L 16V DOHC', '2.4L',
       '3.5L', '3.7L', 'Electric ZEV 499hp', 'Electric Motor',
       '2.0L 16-Valve DOHC VTEC Turbo Engine', '2.0 L', '2.0L DOHC',
       'Engine: Dual Motor -inc: start/stop pushbutton', '3.5 L',
       '3.7 Liter', '3.5', '3.5L 273.0hp', '3.5L 290.0hp', '3.7',
       '3.0L TFSI', 'Electric ZEV', '2.0 Liter Turbo', '2.0L', '1.8L',
       '3.0L', '0.0', '2.9L', 'Supercharged Gas', '0.0L', '5.2L', '2.5L',
       '355.0HP Electric Motor Electric Fuel System',
       'Electric 402hp 490ft. lbs.', '2.0L Turbocharged',
       'Dual Synchronous Electric Motors Engine', 'L Electric Motor',
       '3.0L Supercharged', 'Dual Asynchronous Electric Motors Engine',
       '2.0L TFSI', 'Dual AC Electric Motors', '4.0L', 'Plug-in Hybrid',
       'Not Specified', '2.0-liter TFSI four-cylinder engine',
       '2.0L 252.0hp', '4.2L',
       'Range-Extended Electric 1

In [130]:
#The majority of the above listed are electric or hybrid engines
df[df['engine_cylinders'].isnull()].groupby(['engine'], dropna=False).size().reset_index(name='Count').sort_values(by='Count', ascending=False).head(50)

,engine,Count
513,Gas,3382
412,Electric,3195
558,Turbo Gas,2716
471,Electric Motor,2695
578,–,1115
517,Hybrid,293
539,Plug-in Hybrid,149
409,ELECTRIC MOTOR,128
520,L Electric Motor,74
387,Dual AC Electric Motors,66


In [131]:
df.groupby(['engine_cylinders'], dropna=False).size().reset_index(name='Count')

,engine_cylinders,Count
0,0.0,36
1,2.0,3
2,3.0,3397
3,4.0,62092
4,5.0,589
5,6.0,41684
6,8.0,17054
7,10.0,239
8,12.0,83
9,NaN,16008


#### Engine Horse Power

In [132]:
def get_left_hand_side_hp(text):
    text=text.replace(',','')
    result=""
    already_found_dot=False
    for char in text[::-1]:
        if char.isdigit():
            result = char + result
        elif char == '.' and not already_found_dot:
            result = char + result
            already_found_dot=True   
        else:
            break  # Stop when we encounter a non-number, non-dot character, or a second dot
    return result


def get_engine_HP(text):
    possible_splitters=['HP', 'KW']
    for splitter in possible_splitters:
        for i in range(text.upper().count(splitter)):
            horse_power=get_left_hand_side_hp(text.upper().split(splitter)[i].strip())
            if horse_power.replace('.', '', 1).isdigit() and horse_power.count('.')< 2:
                if splitter=='KW':
                    return float(horse_power)*1.34102
                else: 
                    return float(horse_power)
                
    # Find all the numbers (including decimals) in the string
    numbers = re.findall(r'\d+\.?\d*', text)
    numbers = [float(num) if '.' in num else int(num) for num in numbers]
    sorted_numbers = sorted(numbers, reverse=True)
    for number in sorted_numbers: #We assume that the HP is the greatest number between 60 and 1200 that is not surrounded by a volume unit
        if 60<=number<=500:
            i=text.find(str(number))
            j=i+len(str(number))
            is_HP=True
            possible_other_units = ['CI', 'ci','c.i.','CC', 'cc']
            for unit in possible_other_units:
                if unit in text[j:j+6]:
                    is_HP=False
                    break
                if unit in text[i-6:i]:
                    is_HP=False
                    break
            if is_HP:
                return number

    return None


In [133]:
df['engine_HP']=df['engine'].apply(get_engine_HP)

In [134]:
# Check nulls in engine_HP: Taking a sample of the null values, it seams that non of them have information about the power of the car
df[df['engine_HP'].isnull()]['engine'].unique()[:60]

array(['2.4L I-4 DOHC, i-VTEC variable valve control, premium unleaded,',
       '2L I-4 gasoline direct injection, DOHC, VTEC variable valve cont',
       '3.5L V-6 gasoline direct injection, i-VTEC variable valve contro',
       '1.5L I-4 gasoline direct injection, DOHC, VTEC variable valve co',
       '3.5L V-6 i-VTEC variable valve control, premium unleaded, engine',
       '2.4L I4', '3.5L V6 24V GDI SOHC',
       '3.5L V-6 gasoline direct injection, variable valve control, prem',
       '3.0L PGM-FI 24V SOHC i-VTEC V6 -inc: Variable Cylinder Managemen',
       'Gas', '2.0L I4 16V GDI DOHC Turbo', '3.5L V6 SOHC 24V',
       'Electric', 'Turbo Gas',
       '3L V-6 i-VTEC variable valve control, premium unleaded, engine w',
       '2.4L I-4 gasoline direct injection, DOHC, i-VTEC variable valve',
       '3.5L V6 24V MPFI SOHC',
       '3.2L V-6 variable valve control, premium unleaded, engine with 2',
       '3.5L V-6 variable valve control, premium unleaded, engine with 2',
       

In [135]:
#The number of missing values is really high
df.groupby(['engine_HP'], dropna=False).size().reset_index(name='Count')

,engine_HP,Count
0,60.0,1
1,61.0,1
2,62.0,10
3,63.0,1
4,70.0,22
...,...,...
397,834.0,7
398,845.0,5
399,1000.0,1
400,1020.0,11


#### Fuel type

For the sake of simplicity, we only consider 5 categories: Electric (fuel type not informed), Gasoline, Diesel, Hybrid and Gas (includes E85 and natural gas)

En aras de la simplicidad, consideramos que hay solo cinco categorías de tipo de motorización: eléctrico (cuando el tipo de combustible no está informado), gasolina, diesel, híbrido y gas (incluye E85 y gas natural)

In [136]:
df.loc[df['fuel_type'].apply(lambda x: isinstance(x, float) and np.isnan(x)),'fuel_type']=None # substitue NaN for None
df.loc[df['fuel_type'] == '–', 'fuel_type'] = None


def get_fuel_type(text):
    if text is None:
        return 'Electric'
    if "HYBRI" in text.upper():
        return "Hybrid"
    if "GASOL" in text.upper() or "UNLEADED" in text.upper():
        return "Gasoline"
    if "DIESEL" in text.upper():
        return "Diesel"
    if "GAS" in text.upper() or 'E85' in text.upper():
        return "Gas"
    return None

df['fuel_type_v2']=df['fuel_type'].apply(get_fuel_type)
df['fuel_type'].unique()

array(['Gasoline', None, 'Hybrid', 'E85 Flex Fuel', 'Gas',
       'Plug-In Hybrid', 'Diesel', 'Flexible Fuel',
       'Gasoline / Natural Gas', 'Gasoline fuel type',
       'Gasoline/Mild Electric Hybrid', 'Plug-in Gas/Electric Hybrid',
       'Other', 'Diesel (B20 capable)', 'Unspecified', 'Gaseous', 'Flex',
       'Plug-In Hybrid Fuel', 'Regular Unleaded',
       'Compressed Natural Gas', 'Natural Gas', 'Gasoline Fuel',
       'Premium Unleaded', 'Plug-in Hybrid Electric (PHEV)',
       'PHEV (plug-in hybrid electric vehicle)', 'PHEV Hybrid Fuel',
       'mild', 'MHEV (mild hybrid electric vehicle)',
       'Gasoline/Mild Electric Hybri'], dtype=object)

In [137]:
# Check nulls in fuel_type_v2: All null values in fuel_type_v2 are due to the fact that fuel_type variable doesn't provide the information
df[df['fuel_type_v2'].isnull()]['fuel_type'].unique()

array(['Flexible Fuel', 'Other', 'Unspecified', 'Flex', 'mild'],
      dtype=object)

In [138]:
df.groupby(['fuel_type','fuel_type_v2'], dropna=False).size().reset_index(name='Count')

,fuel_type,fuel_type_v2,Count
0,Compressed Natural Gas,Gas,2
1,Diesel,Diesel,2880
2,Diesel (B20 capable),Diesel,4
3,E85 Flex Fuel,Gas,1492
4,Flex,NaN,1
5,Flexible Fuel,NaN,11
6,Gas,Gas,20
7,Gaseous,Gas,6
8,Gasoline,Gasoline,122148
9,Gasoline / Natural Gas,Gasoline,27


In [139]:
df.head()

,year_manufacture,years,make,model,mileage,stock_type,interior_color,exterior_color,drive_train,fuel_type,...,num_speeds,transmission_type,drive_train_v2,stock_type_v2,model_v2,vehicle_type,engine_displacement,engine_cylinders,engine_HP,fuel_type_v2
0,2006,19,acura,Acura TSX Base,76497,Used,Biege,Green Pearl,Front-wheel Drive,Gasoline,...,NaN,Automatic,Front-wheel Drive,Used,TSX,Compact,2.4,4.0,NaN,Gasoline
1,2021,4,acura,Acura RDX Technology Package,54931,Used,Espresso,Majestic Black Pearl,All-wheel Drive,Gasoline,...,10.0,Automatic,All-wheel Drive,Used,RDX,SUV,2.0,4.0,NaN,Gasoline
2,2017,8,acura,Acura MDX 3.5L w/Technology Package,103720,Used,Ebony,Modern Steel Metallic,All-wheel Drive,Gasoline,...,NaN,Automatic,All-wheel Drive,Used,MDX,SUV,3.5,6.0,NaN,Gasoline
3,2024,1,acura,Acura Integra A-SPEC,17309,Used,Ebony,Platinum White Pearl,Front-wheel Drive,Gasoline,...,NaN,Automatic,Front-wheel Drive,Used,INTEGRA,Compact,1.5,4.0,NaN,Gasoline
4,2017,8,acura,Acura RDX Technology & AcuraWatch Plus Package,66552,Used,Ebony,White Diamond Pearl,All-wheel Drive,Gasoline,...,NaN,Automatic,All-wheel Drive,Used,RDX,SUV,3.5,6.0,NaN,Gasoline


#### Color

In [148]:
ini=0
fin=499
df_list=[]
for _ in range(20):
    path='colors/colors_'+str(ini)+'_'+str(fin)+'.csv'
    df_list.append(pd.read_csv(path, delimiter=';'))
    ini+=500
    fin+=500
df_list.append(pd.read_csv('colors/other_colors.csv', delimiter=';'))

df_color_dictionary=pd.concat(df_list, ignore_index=True,axis=0)[['Color','R','G','B']]

df_auxiliar = pd.DataFrame({'Color': ["'", 'Ebony &#47; Terracotta', 'N/A', 'None', 'null'],
        'R': [None, 80, None, None, None,],
        'G': [None, 40, None, None, None,],
        'B': [None, 20, None, None, None,]})

df_color_dictionary=pd.concat([df_color_dictionary, df_auxiliar], ignore_index=True,axis=0)

In [149]:
print('Number of entries before the left join:', len(df)) 
df_color_dictionary.columns=['interior_color', 'R_interior', 'G_interior', 'B_interior']
df=pd.merge(df, df_color_dictionary, how='left', on='interior_color')
df_color_dictionary.columns=['exterior_color', 'R_exterior', 'G_exterior', 'B_exterior']
df=pd.merge(df, df_color_dictionary, how='left', on='exterior_color')
print('Number of entries after the left join:', len(df))

Number of entries before the left join: 141185
Number of entries after the left join: 141185


In [151]:
color_list=list(set((list(df['interior_color'])+list(df['exterior_color']))))
sorted_color_list=sorted(color_list)
print(len(sorted_color_list))
sorted_color_list[0:500]

9872


["'",
 "''",
 "''''",
 "''westm''",
 "'DEEP BLACK PEARL'",
 "'SILVER'",
 "'SUPER WHT'",
 '(Includes (BT3) Dark Argent  Lower Accent Color.)',
 '(Includes Raven Sapele Real Wood Trim.)',
 '(Includes sueded seat inserts.)',
 '(With Black Olive Ash wood trim.)',
 '(With Brushed Aluminum Trim.)',
 '(With Lunar Brushed Aluminum Trim.)',
 '(With Sapele high-gloss wood trim.)',
 '*',
 '*^',
 '+',
 '-',
 '-SELECT-',
 '-Select-',
 '.',
 '..',
 '/Okapi Brown in',
 '0',
 '00040',
 '0040',
 '0070',
 '0085',
 '0089',
 '01',
 '01G3',
 '01J9',
 '01L5',
 '01L6',
 '01U',
 '01g3',
 '01h1',
 '01j7',
 '01k3',
 '0202',
 '0223',
 '03U5',
 '03U9',
 '03r3 Barcelona',
 '03t3/Ea20',
 '03t5',
 '040',
 '0416',
 '04X4',
 '04u3',
 '06v7',
 '06x3',
 '06x4',
 '0Q',
 '0q',
 '1',
 '101',
 '105',
 '105/ARTICO man-',
 '11',
 '1201: GRAY, 1203 and 1205: DARK BLUE/BLACK, 1207:',
 '12U',
 '130R White',
 '13181',
 '135/Macchiato B',
 '149',
 '161',
 '16U',
 '16u',
 '1WT/1FL-Cloth, Jet Black, Interior Trim',
 '1Wt 1Lt 1Sp 2Lt

In [152]:
# Missing percentage
round((df.isnull().sum()/df.shape[0])*100,2)

year_manufacture        0.00
years                   0.00
make                    0.00
model                   0.00
mileage                 0.00
stock_type              0.00
interior_color          0.00
exterior_color          0.00
drive_train             0.00
fuel_type               6.18
transmission            0.00
engine                  0.00
price_USD               0.00
url                     0.00
mpg_city                9.83
mpg_highway             9.83
num_speeds             64.76
transmission_type       0.47
drive_train_v2          0.96
stock_type_v2           0.00
model_v2                0.00
vehicle_type            0.00
engine_displacement    11.68
engine_cylinders       11.34
engine_HP              94.78
fuel_type_v2            0.03
R_interior_x            0.01
G_interior_x            0.01
B_interior_x            0.01
R_exterior_x            0.00
G_exterior_x            0.00
B_exterior_x            0.00
R_interior_y            0.01
G_interior_y            0.01
B_interior_y  